In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [2]:
t = torch.cuda.get_device_properties(0).total_memory
print("GPU Memory: {}".format(round(t / 1e9, 2)))

GPU Memory: 17.07


In [3]:
!pip install simpletransformers

     |████████████████████████████████| 194kB 3.5MB/s 
     |████████████████████████████████| 675kB 14.0MB/s 
     |████████████████████████████████| 3.8MB 14.9MB/s 
     |████████████████████████████████| 204kB 43.5MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
     |████████████████████████████████| 1.1MB 32.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=49ea85708afd424690557a5f33f4c468dbe12c329d8494970e612e7934073662
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=83f8af6fed9ebbdfaa7583f03e2606f495edd522acfa1b014894ecca6d1c0f86
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [4]:
# !pip install transformers==2.10.0

In [5]:
# !pip install simpletransformers==0.26.0

In [6]:
from simpletransformers.ner import NERModel

In [7]:
le = LabelEncoder()

def enc(sid):
  sid = [str(id_) for id_ in sid]
  return le.fit_transform(sid)

def drop_codes(df):
  return df.drop(['codes'], axis=1), df.codes

In [8]:
train = pd.read_csv('drive/My Drive/data-ner/train_ner.csv', header=0)
train.sentence_id = enc(train.sentence_id)
train, train_codes = drop_codes(train)
print(train.shape)
train.head(5)

(1498953, 4)


,sentence_id,words,labels,country
0,0,APRESENTAÇÃO,O,B
1,0,O,B,B
2,0,Brasil,I,B
3,0,é,I,B
4,0,um,I,B


In [9]:
val = pd.read_csv('drive/My Drive/data-ner/val_ner.csv', header=0)
val.sentence_id = enc(val.sentence_id)
val, val_codes = drop_codes(val)
print(val.shape)
val.head(5)

(318402, 4)


,sentence_id,words,labels,country
0,0,ELEZIONI,O,S
1,0,2018,O,S
2,0,·,O,S
3,0,PROGRAMMA,O,S
4,0,DI,O,S


In [10]:
test = pd.read_csv('drive/My Drive/data-ner/test_ner.csv', header=0)
test.sentence_id = enc(test.sentence_id)
test, test_codes = drop_codes(test)
print(test.shape)
test.head(5)

(318995, 4)


,sentence_id,words,labels,country
0,0,Projeto,O,B
1,0,Brasil,O,B
2,0,Novo,O,B
3,0,Plataforma,O,B
4,0,de,O,B


In [11]:
train.words = train.words.apply(lambda x: str(x))
test.words = test.words.apply(lambda x: str(x))
val.words = val.words.apply(lambda x: str(x))

In [12]:
model = NERModel(#'bert',
                 #'drive/My Drive/bert-lm/',
                 #'bert-base-multilingual-cased',
                 'xlmroberta',
                 #'drive/My Drive/roberta-lm/',
                 'xlm-roberta-base',
                 labels=['B', 'I', 'O'],
                 args={'train_batch_size': 32, 
                        'eval_batch_size': 32,
                        'num_train_epochs': 2,
                        'max_seq_length': 200,
                        'save_steps': 0,
                        'evaluate_during_training': True,
                        'evaluate_during_training_steps': int(train.shape[0] / 32),
                        'evaluate_during_training_verbose': True,
                        'fp16': False,
                        'overwrite_output_dir': True,
                        'reprocess_input_data': True,
                        'learning_rate': 2e-5,
                        'manual_seed':42
                        }
                 )

In [13]:
model.train_model(train_data=train, eval_data=val)

Running loss: 0.304537

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.018102


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.024399


In [14]:
def f_macro(true, preds):
  return metrics.f1_score(true, preds, average='macro')

def f_micro(true, preds):
  return metrics.f1_score(true, preds, average='micro')

def softmax(logits):
  return np.exp(logits) / np.sum(np.exp(logits), axis=0)

In [15]:
result, model_outputs, preds_list = model.eval_model(test, verbose=True)

In [16]:
result

{'eval_loss': 0.06611851085482401,
 'f1_score': 0.7478802092729568,
 'precision': 0.8591264701310813,
 'recall': 0.6621411172782814}

In [17]:
preds = [item for sublist in preds_list for item in sublist]

In [18]:
def pad_preds(sentence, preds):
  print("{} {} -> {}".format(len(sentence), len(preds), len(preds + [preds[-1]] * (len(sentence) - len(preds)))))
  return preds + [preds[-1]] * (len(sentence) - len(preds))

In [19]:
len(preds_list)

9075

In [20]:
test_to_preds = {"sid": [], "words": [], "true": [], "preds": []}
for i, sid in test.groupby(by='sentence_id'):
  if sid.shape[0] != len(preds_list[i]):
    preds_list[i] += preds_list[i][-1] * (sid.shape[0] - len(preds_list[i]))
  test_to_preds["sid"].extend(list(sid.sentence_id.values))
  test_to_preds["words"].extend(list(sid.words.values))
  test_to_preds["true"].extend(list(sid.labels.values))
  test_to_preds["preds"].extend(preds_list[i])

In [21]:
[len(item) for item in test_to_preds.values()]

[318995, 318995, 318995, 318995]

In [22]:
output_df = pd.DataFrame(test_to_preds)

In [23]:
output_df["codes"] = test_codes

In [24]:
output_df["country"] = test.country

In [25]:
output_df.head()

,sid,words,true,preds,codes,country
0,0,Projeto,O,O,999,B
1,0,Brasil,O,O,999,B
2,0,Novo,O,O,999,B
3,0,Plataforma,O,O,999,B
4,0,de,O,O,999,B


In [26]:
output_df.to_csv('outputs.csv', index=False)

In [27]:
output_df.preds.value_counts()

I    301953
B      9597
O      7445
Name: preds, dtype: int64

In [28]:
f_macro(output_df.true, output_df.preds)

0.8818463592710022